<a href="https://colab.research.google.com/github/tylerchi1000/Satander-Customer-Transaction/blob/Test/Satander_Customer_Transaction_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/train.csv')
test = pd.read_csv('/test.csv')


In [ ]:
train.shape

(200000, 202)

In [ ]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [ ]:
train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [ ]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size = 0.20, random_state = 42)

In [ ]:
from tensorflow import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
X_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
153248,12.3039,-8.3899,9.1944,8.0649,9.0247,-1.9559,5.1565,21.1631,2.7437,8.5623,-9.1195,-5.0654,14.0421,9.3000,11.4089,14.7424,5.4342,-9.7680,11.9534,4.5106,13.3095,27.9682,10.3165,2.4899,8.7296,14.2002,3.8301,0.5474,5.2150,4.8237,-24.0381,10.9788,0.2066,20.0695,11.2086,1.4242,5.5014,4.1958,6.8433,0.6589,...,30.6330,5.4488,7.0295,3.4218,1.3661,19.1868,3.3058,-4.5841,3.4557,6.0094,-8.3475,7.9825,13.5424,-0.2908,13.9061,11.8151,3.0716,15.0284,2.1214,8.0848,-4.8168,10.6747,-2.1049,8.8938,19.6247,-2.7233,13.4097,-20.9951,20.2230,-0.3264,5.5185,7.9504,0.9184,5.9945,11.0078,-1.0936,-2.3412,8.1712,12.9046,-1.9309
67802,15.4069,2.7820,9.2951,7.1997,8.5359,-4.5422,5.4210,9.9651,4.0623,8.4689,-6.0590,0.7759,13.7854,4.0456,5.4438,13.8766,8.6089,-16.3946,20.4100,18.2099,11.9410,29.8901,11.0320,3.4647,4.9191,13.5724,-10.3379,-0.5933,5.5332,3.7098,-22.4284,8.6950,0.4795,18.2231,11.6430,-5.9229,5.7317,3.3921,6.5169,-6.0109,...,16.9319,5.5766,4.9744,12.8968,-1.7498,16.7136,3.3190,-4.8081,1.0132,5.9494,-5.5387,2.0530,15.1127,-4.3515,19.6645,13.2448,-8.3029,8.8979,11.6191,2.5894,-2.2567,10.0503,-5.1231,9.7286,-3.4912,5.4957,12.0615,-17.0887,10.5468,0.5903,3.0063,5.6555,2.1527,1.3518,15.4728,0.2686,6.5523,8.4698,22.0454,1.4756
148889,9.6427,-4.6261,6.9610,5.4054,12.0859,-11.2917,4.5290,13.8605,-0.8366,8.4388,-4.8543,-3.8446,13.7602,11.3746,9.2359,14.9431,7.9527,-6.8255,12.5890,21.1809,9.1416,27.5496,4.1218,3.8318,6.1385,14.4584,0.0636,-1.5841,5.1242,7.3442,3.3498,4.8306,-1.3744,18.3092,11.8856,7.3965,3.7448,8.7036,8.1327,5.1036,...,23.6963,5.2485,8.7027,8.3478,-5.7018,19.3036,2.7621,0.7174,3.7536,6.2652,-7.7119,3.1409,17.9489,-6.6449,4.4238,10.9679,-10.2859,8.2506,9.9924,5.9168,-3.7932,11.4330,8.0982,15.4265,7.4456,0.2757,7.5569,-7.1702,15.3692,0.4535,3.4351,9.1779,1.5004,1.9895,20.4072,-0.1118,0.5692,9.3290,12.8980,-9.4318
103093,9.6881,-5.6696,11.2709,8.2812,13.9232,-16.1434,4.9664,20.1092,-5.9868,8.4514,-2.8435,-6.8732,14.2079,6.5918,11.2923,14.1828,10.0402,1.2997,20.5645,28.4864,10.9094,7.2806,5.0493,3.6683,14.8643,13.7662,-9.7314,-1.3767,4.7863,2.0560,-16.1634,13.5568,-1.1293,7.6161,11.9110,2.8845,4.0571,7.9969,10.3673,-1.8724,...,24.3598,6.1042,6.8922,17.9486,0.3322,11.7082,2.8908,-5.6976,8.1320,5.2168,-5.0848,-4.4787,22.2903,4.3004,28.5218,12.3033,3.7143,13.6896,16.9191,2.7819,-8.9304,9.5898,-10.1931,17.1104,25.9923,-12.5229,11.8213,-15.8289,19.5687,1.2158,-4.9494,9.2727,1.1371,3.7435,20.6906,1.3752,7.4442,9.2145,18.2777,-2.5865
104681,7.1128,-2.0830,11.4807,8.3033,10.6180,-6.4743,5.0078,21.0212,-4.9779,9.1260,4.6706,-10.6114,13.7493,6.3089,4.9396,15.0864,11.1357,-10.7133,11.6957,3.6591,18.2128,23.6617,1.6646,2.6451,6.1443,13.9475,-8.1451,-0.6311,5.6320,2.3330,1.0674,13.3107,-2.3119,11.2375,11.9568,7.8714,8.0269,8.7347,8.4093,0.8316,...,21.4064,5.5814,6.0163,17.4770,-1.0434,14.3632,2.5999,-0.3436,5.7521,5.7825,0.8032,-2.7698,13.2530,-0.1776,15.6257,17.1736,0.6404,11.1395,15.4044,2.4522,-9.5583,8.9755,-0.8453,8.9209,20.0507,-9.1914,5.8473,-7.1620,19.8325,2.1774,7.3583,8.1992,1.3436,8.8929,21.6711,-2.0557,6.4975,8.3110,13.7728,-5.9028


In [ ]:
min_max_scaler = MinMaxScaler()
X_train[X_train.columns] = min_max_scaler.fit_transform(X_train[X_train.columns])

In [ ]:
X_test[X_test.columns] = min_max_scaler.transform(X_test[X_test.columns])

In [ ]:
test_features[train_features.columns] = min_max_scaler.transform(test_features[test_features.columns])

In [ ]:
X_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
153248,0.596666,0.261741,0.428019,0.612700,0.340608,0.614417,0.467553,0.707785,0.641397,0.639518,0.270881,0.485855,0.497992,0.518071,0.743709,0.581457,0.274740,0.450562,0.431533,0.355425,0.510421,0.642687,0.784732,0.349239,0.388691,0.789905,0.703268,0.713525,0.412198,0.523200,0.213287,0.493575,0.581675,0.668977,0.416253,0.471520,0.608598,0.412288,0.364496,0.499690,...,0.578395,0.390717,0.673557,0.280448,0.665187,0.445310,0.602123,0.544198,0.440357,0.606474,0.197148,0.754068,0.384282,0.539800,0.363142,0.486769,0.652087,0.659612,0.531083,0.752465,0.495017,0.580538,0.437584,0.480551,0.619023,0.558166,0.663611,0.379340,0.634509,0.338155,0.602806,0.549233,0.387406,0.591312,0.120274,0.437114,0.364832,0.356740,0.325091,0.556163
67802,0.752892,0.701230,0.434109,0.547296,0.298458,0.562498,0.511576,0.206579,0.705230,0.626509,0.351180,0.620809,0.287565,0.323754,0.310472,0.258168,0.458697,0.324115,0.592881,0.643314,0.473151,0.675269,0.820736,0.615185,0.231261,0.453518,0.348342,0.592302,0.466485,0.459360,0.245194,0.366035,0.599024,0.601124,0.546181,0.278391,0.618906,0.350688,0.355470,0.272118,...,0.370318,0.481143,0.457487,0.562838,0.568170,0.373655,0.607865,0.540250,0.316812,0.584603,0.287065,0.607286,0.408424,0.442197,0.486867,0.567940,0.421655,0.258299,0.691748,0.493063,0.568609,0.510117,0.382315,0.509829,0.243593,0.789162,0.609163,0.434718,0.246187,0.477293,0.525588,0.430794,0.488438,0.436890,0.352410,0.579993,0.638217,0.406901,0.793360,0.607476
148889,0.462683,0.409804,0.292948,0.411657,0.604582,0.427005,0.363114,0.380931,0.468073,0.622317,0.382787,0.514060,0.266907,0.594794,0.585887,0.656398,0.420674,0.506710,0.443660,0.705749,0.396911,0.635590,0.473016,0.715338,0.281640,0.928254,0.608913,0.487008,0.396707,0.667654,0.756160,0.150225,0.481170,0.604288,0.618741,0.628510,0.529970,0.757787,0.400154,0.651341,...,0.473048,0.248992,0.849474,0.427261,0.445119,0.448694,0.365598,0.637638,0.455425,0.699716,0.217495,0.634217,0.452027,0.387072,0.159406,0.438669,0.381482,0.215926,0.664231,0.650128,0.524441,0.666061,0.624422,0.709663,0.421220,0.642453,0.427244,0.575327,0.439718,0.456530,0.538768,0.612583,0.435045,0.458101,0.608951,0.540094,0.454298,0.551236,0.324753,0.443175
103093,0.464969,0.368754,0.553601,0.629051,0.763017,0.329609,0.435914,0.660614,0.218751,0.624072,0.435545,0.444089,0.633904,0.417917,0.735240,0.372503,0.541633,0.661753,0.595829,0.859272,0.445056,0.291959,0.519688,0.670732,0.642146,0.557359,0.363536,0.509049,0.339060,0.364578,0.369376,0.637545,0.496752,0.211328,0.626338,0.509906,0.543949,0.703622,0.461950,0.413322,...,0.483124,0.854454,0.659121,0.713401,0.632995,0.228636,0.421586,0.524572,0.676891,0.317562,0.301595,0.445597,0.518772,0.650155,0.677174,0.514486,0.665107,0.571972,0.781405,0.502150,0.376769,0.458180,0.289474,0.768720,0.722440,0.282748,0.599463,0.452578,0.608251,0.572233,0.281055,0.617476,0.405307,0.516441,0.623685,0.696064,0.665634,0.532002,0.600346,0.546288
104681,0.335310,0.509847,0.566290,0.630722,0.478002,0.523712,0.442804,0.701434,0.267592,0.718026,0.632692,0.357724,0.257972,0.407455,0.273853,0.709906,0.605112,0.432524,0.426616,0.337531,0.643960,0.569676,0.349370,0.391581,0.281880,0.654504,0.403275,0.588285,0.483340,0.380453,0.710919,0.623801,0.421573,0.344411,0.640037,0.640994,0.721643,0.760171,0.407803,0.505582,...,0.438271,0.484540,0.567031,0.699346,0.590164,0.305558,0.295036,0.618938,0.556512,0.523766,0.490084,0.487900,0.3

In [ ]:
#Large Number of Unkown Features. Investigated PCA to see if feature space could
#be reduced.

'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_scorerfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))
'''

In [ ]:
#The linear line shows that there isn't a good way to break down the variables
#into principle components. It appears some transformation may have already been
#applied
'''

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCApca_test = PCA(n_components=50)
pca_test.fit(X_train_scaled)sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=10, ymin=0, ymax=1)
display(plt.show())evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(10))
'''

Below is a tensorflow model

In [ ]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
def create_base_nn(metrics, input_size):

  model = Sequential()
  model.add(Dense(100, activation='relu', input_dim=input_size))
  model.add(Dropout(rate=0.2))
  model.add(Dense(100, activation = 'relu'))
  model.add(Dropout(rate=0.2))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics = metrics)
  
  #model.compile(loss = 'binary_crossentropy', 
  #              optimizer = 'adam', 
  #              metrics = ['accuracy'])
  return model

In [ ]:
model2 = create_base_nn(metrics = METRICS, input_size=200)

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               40200     
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 60,401
Trainable params: 60,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
result = model2.fit(X_train, y_train, batch_size = 100, epochs = 10, validation_data=(X_test, y_test))

Epoch 1/10
1600/1600 [==============================] - 13s 7ms/step - loss: 0.2735 - accuracy: 0.9051 - precision: 0.7168 - auc: 0.7776 - val_loss: 0.2592 - val_accuracy: 0.9043 - val_precision: 0.8281 - val_auc: 0.8539
Epoch 2/10
1600/1600 [==============================] - 10s 6ms/step - loss: 0.2385 - accuracy: 0.9127 - precision: 0.6765 - auc: 0.8480 - val_loss: 0.2500 - val_accuracy: 0.9071 - val_precision: 0.8124 - val_auc: 0.8585
Epoch 3/10
1600/1600 [==============================] - 11s 7ms/step - loss: 0.2359 - accuracy: 0.9126 - precision: 0.6614 - auc: 0.8526 - val_loss: 0.2383 - val_accuracy: 0.9121 - val_precision: 0.7528 - val_auc: 0.8594
Epoch 4/10
1600/1600 [==============================] - 7s 4ms/step - loss: 0.2349 - accuracy: 0.9138 - precision: 0.6722 - auc: 0.8539 - val_loss: 0.2361 - val_accuracy: 0.9125 - val_precision: 0.6547 - val_auc: 0.8598
Epoch 5/10
1600/1600 [==============================] - 6s 4ms/step - loss: 0.2348 - accuracy: 0.9135 - precision: 0.

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = model2.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8601320867060298

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')


In [ ]:
test_features.shape

(200000, 200)

In [ ]:
id_code_test = test['ID_code']
pred = model2.predict(test_features)
pred_ = pred[:,0]
pred_

array([0.23863569, 0.41215286, 0.06348121, ..., 0.00694194, 0.08110669,
       0.21699587], dtype=float32)

In [ ]:
submission = pd.DataFrame({'ID_code': id_code_test, 'target' : pred_})

In [ ]:
from google.colab import files

submission.to_csv('output.csv', encoding = 'utf-8-sig', index=False) 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
# Some feature Enginering Ideas from Aladdin Persson https://www.youtube.com/watch?v=MOnk75_8b9M&list=PLhhyoLH6IjfxkVb3Yx4mjLdwMWP5S_Hct&index=5
'''As investigated above in PCA there is no corrleation among the features. There appears to have been some transformations
to the data. Feature engineering will hopefully allow the neural network to expose some interesting connections in the data'''



,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [ ]:
#Half of the train data appeared to be copies of other examples.
#Take out unique examples and create a binary column saying if it is unique.
from tqdm import tqdm
col_names = [f"var_{i}" for i in range(200)]
for col in tqdm(col_names):
  count = test[col].value_counts()
  #If count has only 1 then it is unique. Check for all.
  uniques = count.index[count == 1]
  test[col + "_u"] = test[col].isin(uniques)

test["has_unique"] = test[[col + "_u" for col in col_names]].any(axis=1)

In [ ]:
test["has_unique"].sum()

In [ ]:
real_test = test.loc[test["has_unique"], ["ID_code"] + col_names]
fake_test = test.loc[~test["has_unique"], ["ID_code"] + col_names]

#Original Author concatinated train and test set. This exposes the test set

train_and_test = pd.concat([train, real_test], axis=0)


In [ ]:
for col in tqdm(col_names):
  count = train_and_test[col].value_counts().to_dict()
  train_and_test[col + "_unique"] = train_and_test[col].apply(
      lambda x: 1 if count[x] == 1 else 0).values
      #apply each value to those and map to new column
    fake_test[col+"_unique"] = 0
  )

In [ ]:
real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(["target"], axis = 1, inplace = True)
engineered_train = train_and_test[train_and_test["ID_code"].str.contains("train")].copy()

In [ ]:
engineered_test = pd.concat([real_test, fake_test], axis = 0)
